## Imports


In [107]:
import numpy as np
from icecream import ic
from gxgp import  Node
from tqdm import tqdm
import operator
import inspect

In [108]:
a = np.array([1])

In [109]:
def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5
def true_f2(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5 + x[0] * x[2] / 10

In [110]:
def my_f(x: np.ndarray) -> np.ndarray:
    return np.minimum(np.divide(np.maximum(np.maximum(x[1],x[2]),np.absolute(435)),np.sqrt(385)),x[0])

In [111]:
def mse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return np.mean((y_true - y_pred) ** 2)

In [112]:
TEST_SIZE = 10_000
TRAIN_SIZE = 1000

def gen_problem():
    x_validation = np.vstack(
        [
            np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
            np.random.random_sample(size=TEST_SIZE) * 2 - 1,
        ]
    )
    y_validation = true_f(x_validation)
    train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
    x_train = x_validation[:, train_indexes]
    y_train = y_validation[train_indexes]
    assert np.all(y_train == true_f(x_train)), "D'ho"
    return x_train, y_train
def gen_problem2():
    x_validation = np.vstack(
        [
            np.random.random_sample(size=TEST_SIZE) * np.exp(3)-np.exp(1),
            np.random.random_sample(size=TEST_SIZE) * 2 - 1,
            np.random.random_sample(size=TEST_SIZE) * 57 - 1,
        ]
    )
    y_validation = true_f2(x_validation)
    train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
    x_train = x_validation[:, train_indexes]
    y_train = y_validation[train_indexes]
    assert np.all(y_train == true_f2(x_train)), "D'ho"
    return x_train, y_train

# np.savez('problem_0.npz', x=x_train, y=y_train)
# problem = np.load('problem_0.npz')
# x = problem['x']
# y = problem['y']

problem = gen_problem2()
x = problem[0]
y = problem[1]

number_vars = x.shape[0]
# compute mse with truef as reference
m_s_e = mse(my_f(x), y)
m_s_e



np.float64(845.0166239907486)

### Getting numpy functions

In [113]:
def get_np_functions()->list[tuple[np.ufunc,str,int]]:
    # Get all attributes of numpy module
    all_attrs = dir(np)

    # Filter to include only ufunc objects
    ufuncs = [getattr(np, attr) for attr in all_attrs if isinstance(getattr(np, attr), np.ufunc)]
    ufunc_list = ufuncs
    # Filter to include only common ufuncs
    # ufunc_list = [(ufunc,ufunc.__name__,ufunc.nin) for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    # ufunc_list = [ufunc for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    ufunc_list = [ufunc for ufunc in ufuncs if ufunc.__name__ in ['absolute','absolute','arccos','arccosh','add','arccos','arccosh','arcsin','arcsinh','arctan','arctan2','arctanh','arcsin','arcsinh','arctan','arctan2','arctanh','ceil','cos','cosh','exp','exp2','log','log10','log1p','log2','logaddexp','logaddexp2','maximum','minimum','multiply','power','sin','sinh','sqrt','square','subtract','tan','tanh','divide']]
    return ufunc_list
ufunc_list = get_np_functions()
[ufunc_list.__name__ for ufunc_list in ufunc_list]


['absolute',
 'absolute',
 'arccos',
 'arccosh',
 'add',
 'arccos',
 'arccosh',
 'arcsin',
 'arcsinh',
 'arctan',
 'arctan2',
 'arctanh',
 'arcsin',
 'arcsinh',
 'arctan',
 'arctan2',
 'arctanh',
 'ceil',
 'cos',
 'cosh',
 'divide',
 'exp',
 'exp2',
 'log',
 'log10',
 'log1p',
 'log2',
 'logaddexp',
 'logaddexp2',
 'maximum',
 'minimum',
 'multiply',
 'power',
 'power',
 'sin',
 'sinh',
 'sqrt',
 'square',
 'subtract',
 'tan',
 'tanh',
 'divide']

## Symbolic Regressor


### Utilify Functions


In [114]:
from tree4 import FastTBTree
def RHH_Init():
    pass
def fitness(tree:FastTBTree,x_train:np.ndarray,y_train:np.ndarray)->float:
    res = [tree.evalute_tree(x) for x in x_train.T]
    return np.mean((res-y_train)**2)
def fitness_proportional_selection(trees:list[FastTBTree],n_elems=2)->list[FastTBTree]:
    """
    Windowing version of fitness proportional selection
    Each tree has a probability of (fitness-min(fitnesses))/(sum(fitnesses)-min(fitnesses)) to be chosen
    """
    fitnesses = np.array([tree.fitness for tree in trees])
    # print(fitnesses)
    fitnesses = fitnesses - np.min(fitnesses)
    fitnesses = fitnesses / np.sum(fitnesses)
  
    return np.random.choice(trees, n_elems, p=fitnesses)
def tournament_selection(trees:list[FastTBTree],n_elems=2)->list[FastTBTree]:
    """
    Tournament selection
    """
    challengers = np.random.choice(trees, n_elems)
    return sorted(challengers, key=lambda x: x.fitness)[0]


def mutation(tree:FastTBTree,max_const:int,md:int)->FastTBTree:
    """
    Mutation of a tree
    Randomly select a mutation method and apply it
    """
    mutation_kind = np.random.randint(5)
    match mutation_kind:
        case 0:
            tree.point_mutation(max_const=max_const)
            return tree
        case 1:
            tree.expansion_mutation(md)
            return tree
        case 2:
            tree.hoist_mutation()
            return tree
        case 3:
            tree.subtree_mutation(md)
            return tree
        case 4:
            tree.collapse_subtree_mutation()
            return tree




### Regression 

In [115]:
N_VARS = number_vars
MD = 6 # Maximum depth
MAX_CONST = 1000 # Maximum constant value
OFFSPRING_SIZE = 20
GENERATIONS = 100
PM = 0.05
NAN_PENALTY = 1000
FIX_NAN = True

operator_list = get_np_functions()
FastTBTree.set_params(operator_list,N_VARS,MAX_CONST)
# RHH initialization
population = [FastTBTree.generate_random_tree_growfull(True,MD) for _ in range(OFFSPRING_SIZE)]
# population.extend([FastTBTree.generate_random_tree_growfull(False,MD*2) for _ in range(OFFSPRING_SIZE)])

best_solution = population[0]

for elem in population:
    elem.fitness = fitness(elem,x,y)
    if FIX_NAN and np.isnan(elem.fitness) or np.isinf(elem.fitness):
        elem.fitness = np.float64(NAN_PENALTY)
    # ic(elem.fitness)
offspring = []
for generation in tqdm(range(GENERATIONS)):
    offspring = []
    # parent selection
    # parents = fitness_proportional_selection(population)
    parents = [tournament_selection(population) for _ in range(2)]
    # recombination
    # for _ in range(OFFSPRING_SIZE):
        # o1,o2 = recombination(parents[0],parents[1])
    o1,o2 = FastTBTree.recombination(parents[0],parents[1])
    offspring.append(o1)
    offspring.append(o2)

    # if np.random.rand() < PM:
    #     offspring_idx = np.random.randint(OFFSPRING_SIZE)
    #     mutation(population[offspring_idx],MAX_CONST,MD)

    # mutation
    

    # survivor selection
    population.extend(offspring)
    for elem in population:
        elem.fitness = fitness(elem,x,y)
        if FIX_NAN and np.isnan(elem.fitness)  or np.isinf(elem.fitness):
            elem.fitness = np.float64(NAN_PENALTY)
    population.sort(key=operator.attrgetter('fitness'))
    population = population[:OFFSPRING_SIZE]
    if population[0].fitness < best_solution.fitness:
        best_solution = population[0]
best_solution


# ic([elem for elem in population if not np.isnan(elem.fitness) and not  np.isinf(elem.fitness)])

c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: invalid value encountered in tan
  return node.data(FastTBTree._evalute_tree(tree, FastTBTree.get_lchild_idx(i), x))
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\denni\AppData\Local\Temp\ipykernel_15896\1202639717.py:6: RuntimeWarning: overflow encountered in square
  return np.mean((res-y_train)**2)
100%|██████████| 100/100 [00:25<00:00,  3.92it/s]


In [116]:
ic([elem.fitness for elem in population])
# ic([elem.to_np_formula() for elem in population])
# print(population[0].print_tree())
best_solution.to_np_formula()
population[0].to_np_formula()



ic| [elem.fitness for elem in population]: [np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                                            np.float64(694.2573550144789),
                         

'30'